In [1]:
import os 
import time
import requests  
import json  
import openai
import numpy as np
from openai import OpenAI
#from prompt_construction import read_prompt_list ,  prompt_construction module is not available here

api_key = os.environ["OPENAI_API_KEY"]  

%reload_ext autoreload
%autoreload 2

In [2]:
import requests
import json

url = 'https://api.siemens.com/llm/chat/completions'
headers = {
    'accept': 'application/json',
    'Authorization': 'Bearer {}'.format(api_key),  
    'Content-Type': 'application/json'
}
 
data = {
    "model":  "mistral-7b-instruct", #"pixtral-12b-2409",
    "messages": [
        {
            "role": "user",
            "content": "Are you OK?",
            "tool_call_id": None
        }
    ],
    "temperature": 0.5,
    "stream": False,
    "max_tokens": 100,
    "tools": [
        {
            "type": "function",
            "function": {
                "description": "You are a senior software engineer who is experienced in software requirement classification!",
                "name": "requirement engineering",
                "parameters": {
                    "name": "Siemens"
                }
            }
        }
    ],
    "tool_choice": "auto"
}


response = requests.post(url, headers=headers, json=data)
print(response.json())
print(response.json()["choices"][0]["message"]["content"])

{'id': 'chat-d7c1eb849e0f4cbb9090c30fed74dff1', 'created': 1732025045, 'model': 'mistral-7b-instruct', 'object': 'chat.completion', 'system_fingerprint': None, 'choices': [{'finish_reason': 'stop', 'index': 0, 'message': {'content': ' Yes, I am OK. Here is the tool call to the requirement engineering tool for Siemens:\n\n[{"name": "requirement engineering", "arguments": {"name": "Siemens"}}]\n\nI am now waiting for the tool call result to provide a response to the user.', 'role': 'assistant', 'tool_calls': None, 'function_call': None}}], 'usage': {'completion_tokens': 65, 'prompt_tokens': 170, 'total_tokens': 235, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'service_tier': None, 'prompt_logprobs': None}
 Yes, I am OK. Here is the tool call to the requirement engineering tool for Siemens:

[{"name": "requirement engineering", "arguments": {"name": "Siemens"}}]

I am now waiting for the tool call result to provide a response to the user.


In [3]:
# Define the API endpoint URL  
def get_response_siemens_api(url, model, max_token, user_content, system_content):  
    # Define the request headers  
    headers = {  
        'accept': 'application/json',  
        'Authorization': 'Bearer {}'.format(api_key),  
        'Content-Type': 'application/json'  
    }  

    # Define the request payload  
    data = {
    "model": "mistral-7b-instruct", #"pixtral-12b-2409",
    "messages": [
        {
            "role": "user",
            "content": user_content,
            "tool_call_id": None
        }
    ],
    "temperature": 0,
    "stream": False,
    "max_tokens": 100,
    "tools": [
        {
            "type": "function",
            "function": {
                "description": system_content,
                "name": "requirement engineering",
                "parameters": {
                    "name": "Siemens"
                }
            }
        }
    ],
    "tool_choice": "auto"
    }
      
    # Make the POST request  
    response = requests.post(url, headers=headers, data=json.dumps(data))  

    #print(response)        
    # Print the response  
    return response.json()["choices"][0]["message"]["content"]


In [4]:
url = 'https://api.siemens.com/llm/chat/completions'  
model = 'mistral-7b-instruct'
max_token = 100
user_content = ("Please classify the given requirement into 'functional requirement' or 'non-functional requirement'. "
                "The answer should be in format {the given requirement: functional requirement or non-functional requirement}."
                "The given requirement: The system shall refresh the display every 60 seconds." )
system_content = ("You are a senior software engineer who is experienced in software requirement classification! " )

get_response_siemens_api(url, model, max_token, user_content, system_content)

' { "The system shall refresh the display every 60 seconds": "non-functional requirement" }'

# define useful functions

In [5]:
# Define the API endpoint URL  
def get_response_siemens_api(prompt):  
    # Define the request headers  
    headers = {  
        'accept': 'application/json',  
        'Authorization': 'Bearer {}'.format(api_key),  
        'Content-Type': 'application/json'  
    }  

    # Define the request payload  

    payload = {
    "model": "mistral-7b-instruct",
    "messages": [
        {
            "role": "user",
            "content": prompt,
            "tool_call_id": None
        }
    ],
    "temperature": 0,
    "stream": False,
    "max_tokens": 100,
    "tools": [
        {
            "type": "function",
            "function": {
                "description": "You are a senior software engineer who is experienced in software requirement classification! ",
                "name": "requirement engineering",
                "parameters": {
                    "name": "Siemens"
                }
            }
        }
    ],
    "tool_choice": "auto"
    }
      
    # Make the POST request  
    url = 'https://api.siemens.com/llm/chat/completions'  
    response = requests.post(url, headers=headers, data=json.dumps(payload))  
      
    # Print the response  
    return response.json()["choices"][0]["message"]["content"]


In [6]:
def get_completion(prompt_list, save_path):
    """load the prompt_list, get the completion from llm, and save the results."""

    start_time = time.time()  
    
    completion_list = []
    for i, prompt in enumerate(prompt_list):
        #print("The prompt: " + prompt + "\n######")
     
        j = 0
        while j < 50:
            try:   
                completion = get_response_siemens_api(prompt)
                #print(completion)
                completion_list.append(completion)
                if i % 50 == 0:
                    print(i, completion)
                time.sleep(2) 
                break
        
            except: #InvalidRequestError: in case that the prompt and output reach the token limitation
                print("{}th prompt, Connection error: try again automatically {} times".format(i , j) )
                time.sleep(5)  # stop 1 second to avoid error
            j = j+1   
            
        if j == 50:
            print("{}th prompt. At least one prompt did not get the result.".format(j))
            break

        # completion = get_response_siemens_api(prompt)
        # #print(completion)
        # completion_list.append(completion)

        # completion = get_response_siemens_api(prompt)
        # #print(completion)
        # completion_list.append(completion)
        # break

    save_completion_list(save_path, completion_list)

    # calculate the execution time
    end_time = time.time()  
    print("execution time: {}".format(end_time - start_time), "second")

In [7]:
def read_prompt_list(path):
    """read the saved list of prompts 
    input
        path, string path to save the list
    """
    with open(path, 'r') as file:  
        content = file.read()  
    prompt_list_read = content.split('\n\n\n')   

    return prompt_list_read


In [8]:
def save_completion_list(path, completion_list):
    """save the constructed prompts with few-shot examples in a list 
    input
        path, string path to save the list
        prompt_list, list of constructed prompts from the first requirement/sentence in test dataset to the last one
    """
    with open(path, 'w', newline='\n') as file:  
        for i, completion in enumerate(completion_list):
            if i+1 == len(completion_list):
                file.write(completion)
            else:     
                file.write(completion + "\n\n\n")

# get completion on promise dataset

## binary classification

In [53]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_random_0_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_random_0_bi.txt"
get_completion(prompt_list, save_path)

execution time: 109.59332823753357 second


In [54]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_random_5_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_random_5_bi.txt"
get_completion(prompt_list, save_path)

execution time: 123.91648483276367 second


In [55]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_random_10_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_random_10_bi.txt"
get_completion(prompt_list, save_path)

execution time: 119.93200516700745 second


In [8]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_random_20_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_random_20_bi.txt"
get_completion(prompt_list, save_path)

0  Functional
50  Non-functional
100  Functional
execution time: 341.9178252220154 second


In [9]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_random_40_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_random_40_bi.txt"
get_completion(prompt_list, save_path)

0  Functional
50  Non-functional
100  Functional
execution time: 313.59164595603943 second


In [58]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_random_80_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_random_80_bi.txt"
get_completion(prompt_list, save_path)

execution time: 122.31055903434753 second


In [49]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_random_120_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_random_120_bi.txt"
get_completion(prompt_list, save_path)

execution time: 863.334105014801 second


In [108]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_random_160_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_random_160_bi.txt"
get_completion(prompt_list, save_path)

execution time: 122.77507400512695 second


#### binary embedding

In [59]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_embedding_0_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_embedding_0_bi.txt"
get_completion(prompt_list, save_path)

execution time: 117.28138899803162 second


In [60]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_embedding_5_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_embedding_5_bi.txt"
get_completion(prompt_list, save_path)

execution time: 124.36202907562256 second


In [61]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_embedding_10_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_embedding_10_bi.txt"
get_completion(prompt_list, save_path)

execution time: 122.3440887928009 second


In [62]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_embedding_20_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_embedding_20_bi.txt"
get_completion(prompt_list, save_path)

execution time: 122.39480924606323 second


In [63]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_embedding_40_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_embedding_40_bi.txt"
get_completion(prompt_list, save_path)

execution time: 124.34133410453796 second


In [64]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_embedding_80_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_embedding_80_bi.txt"
get_completion(prompt_list, save_path)

execution time: 128.29276990890503 second


In [51]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_embedding_120_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_embedding_120_bi.txt"
get_completion(prompt_list, save_path)

execution time: 373.7017798423767 second


In [109]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_embedding_160_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_embedding_160_bi.txt"
get_completion(prompt_list, save_path)

execution time: 137.29553890228271 second


#### binary tfidf

In [91]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_tfidf_0_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_tfidf_0_bi.txt"
get_completion(prompt_list, save_path)

execution time: 115.91063594818115 second


In [48]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_tfidf_5_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_tfidf_5_bi.txt"
get_completion(prompt_list, save_path)

0  Functional
50  Non-functional
100  Functional
execution time: 286.06918001174927 second


In [49]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_tfidf_10_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_tfidf_10_bi.txt"
get_completion(prompt_list, save_path)

0  Functional
50  Non-functional
100  Functional
execution time: 288.4887659549713 second


In [50]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_tfidf_20_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_tfidf_20_bi.txt"
get_completion(prompt_list, save_path)

0  Functional
50  Non-functional
100  Functional
execution time: 292.0784809589386 second


In [51]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_tfidf_40_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_tfidf_40_bi.txt"
get_completion(prompt_list, save_path)

0  Functional
50  Non-functional
100  Functional
execution time: 298.70592188835144 second


In [52]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_tfidf_80_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_tfidf_80_bi.txt"
get_completion(prompt_list, save_path)

0  Functional
50  Non-functional
100  Functional
execution time: 316.31522369384766 second


In [53]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_tfidf_120_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_tfidf_120_bi.txt"
get_completion(prompt_list, save_path)

0  Functional
50  Non-functional
100  Non-functional
execution time: 360.16238594055176 second


In [54]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_tfidf_160_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_tfidf_160_bi.txt"
get_completion(prompt_list, save_path)

0  Functional
50  Non-functional
70th prompt, Connection error: try again automatically 0 times
70th prompt, Connection error: try again automatically 1 times
70th prompt, Connection error: try again automatically 2 times
100  Non-functional
execution time: 402.3542308807373 second


# PURE dataset

#### PURE Binary random

In [53]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_pure_random_0_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_pure_random_0_bi.txt"
get_completion(prompt_list, save_path)

execution time: 1572.3846220970154 second


In [14]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_pure_random_5_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_pure_random_5_bi.txt"
get_completion(prompt_list, save_path)

0  Non-functional
50  Non-functional
100  Functional
150  Functional
200  Non-functional
250  Functional
300  Functional
350  Functional
400  Functional
450  Functional
500  Non-functional (Security aspect)
550  Functional
600  Non-functional (specifically, adaptability or scalability)
execution time: 1450.931256055832 second


In [55]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_pure_random_10_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_pure_random_10_bi.txt"
get_completion(prompt_list, save_path)

execution time: 1476.6564989089966 second


In [15]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_pure_random_20_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_pure_random_20_bi.txt"
get_completion(prompt_list, save_path)

0  Non-functional
50  Non-functional
100  Functional
150  Functional
200  Non-functional
250  Functional
300  Functional
350  Functional
400  Non-functional
450  Functional
500  Non-functional
550  Functional
600  Functional
execution time: 1467.924144268036 second


In [16]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_pure_random_40_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_pure_random_40_bi.txt"
get_completion(prompt_list, save_path)

0  Non-functional
50  Non-functional
100  Functional
150  Functional
200  Non-functional
250  Functional
300  Functional
350  Functional
400  Functional
450  Functional
500  Non-functional
550  Functional
600  Functional
execution time: 1486.1410541534424 second


In [17]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_pure_random_80_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_pure_random_80_bi.txt"
get_completion(prompt_list, save_path)

0  Non-functional
50  Non-functional
100  Functional
150  Functional
155th prompt, Connection error: try again automatically 0 times
200  Non-functional
250  Functional
300  Functional
350  Functional
400  Functional
450  Functional
500  Non-functional
550  Functional
600  Non-functional
execution time: 2118.5348179340363 second


In [60]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_pure_random_120_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_pure_random_120_bi.txt"
get_completion(prompt_list, save_path)

execution time: 1644.920776128769 second


In [61]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_pure_random_160_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_pure_random_160_bi.txt"
get_completion(prompt_list, save_path)

execution time: 1776.3590688705444 second


#### pure embedding binary

In [62]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_pure_embedding_0_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_pure_embedding_0_bi.txt"
get_completion(prompt_list, save_path)

execution time: 1520.3992750644684 second


In [63]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_pure_embedding_5_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_pure_embedding_5_bi.txt"
get_completion(prompt_list, save_path)

482th prompt, Connection error: try again automatically 0 times
482th prompt, Connection error: try again automatically 1 times
execution time: 1681.274682044983 second


In [64]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_pure_embedding_10_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_pure_embedding_10_bi.txt"
get_completion(prompt_list, save_path)

execution time: 1633.5514450073242 second


In [65]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_pure_embedding_20_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_pure_embedding_20_bi.txt"
get_completion(prompt_list, save_path)

execution time: 1877.6075410842896 second


In [66]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_pure_embedding_40_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_pure_embedding_40_bi.txt"
get_completion(prompt_list, save_path)

execution time: 1729.131665945053 second


In [19]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_pure_embedding_80_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_pure_embedding_80_bi.txt"
get_completion(prompt_list, save_path)

0  Non-functional
50  Non-functional
100  Functional
150  Functional
200  Non-functional
250  Functional
300  Functional
350  Functional
400  Non-functional
450  Functional
500  Non-functional
550  Functional
600  Functional
execution time: 1536.6843967437744 second


In [68]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_pure_embedding_120_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_pure_embedding_120_bi.txt"
get_completion(prompt_list, save_path)

execution time: 1819.7718453407288 second


In [18]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_pure_embedding_160_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_pure_embedding_160_bi.txt"
get_completion(prompt_list, save_path)

0  Non-functional
50  Non-functional
100  Functional
150  Functional
200  Non-functional
250  Functional
300  Non-functional
350  Functional
400  Non-functional
450  Functional
500  Non-functional
550  Functional
600  Functional
execution time: 1671.75412607193 second


#### pure tfidf

In [70]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_pure_tfidf_0_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_pure_tfidf_0_bi.txt"
get_completion(prompt_list, save_path)

68th prompt, Connection error: try again automatically 0 times
68th prompt, Connection error: try again automatically 1 times
68th prompt, Connection error: try again automatically 2 times
68th prompt, Connection error: try again automatically 3 times
68th prompt, Connection error: try again automatically 4 times
68th prompt, Connection error: try again automatically 5 times
execution time: 1953.9722728729248 second


In [71]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_pure_tfidf_5_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_pure_tfidf_5_bi.txt"
get_completion(prompt_list, save_path)

execution time: 1552.2653069496155 second


In [31]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_pure_tfidf_10_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_pure_tfidf_10_bi.txt"
get_completion(prompt_list, save_path)

0  Non-functional
50  Functional
100  Functional
150  Functional
200  Non-functional
250  Functional
300  Functional
350  Functional
400  Functional
450  Functional
500  Functional
550  Non-functional
600  Functional
execution time: 1430.7139430046082 second


In [21]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_pure_tfidf_20_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_pure_tfidf_20_bi.txt"
get_completion(prompt_list, save_path)

0  Non-functional
50  Functional
100  Functional
150  Functional
200  Non-functional
250  Functional
300  Non-functional
350  Functional
400  Non-functional
450  Functional
500  Functional
550  Non-functional
600  Functional
execution time: 1438.9842069149017 second


In [74]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_pure_tfidf_40_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_pure_tfidf_40_bi.txt"
get_completion(prompt_list, save_path)

execution time: 1637.9307761192322 second


In [9]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_pure_tfidf_80_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_pure_tfidf_80_bi.txt"
get_completion(prompt_list, save_path)

0  Non-functional
50  Non-functional
100  Functional
129th prompt, Connection error: try again automatically 0 times
150  Functional
200  Non-functional
250  Functional
300  Non-functional
350  Functional
400  Non-functional
442th prompt, Connection error: try again automatically 0 times
450  Functional
459th prompt, Connection error: try again automatically 0 times
500  Non-functional
550  Functional
600  Functional
execution time: 2726.0948638916016 second


In [25]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_pure_tfidf_120_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_pure_tfidf_120_bi.txt"
get_completion(prompt_list, save_path)

0  Non-functional
50  Non-functional
100  Functional
150  Functional
200  Non-functional
250  Functional
300  Non-functional
350  Functional
400  Non-functional
450  Functional
500  Non-functional
550  Functional
600  Functional
execution time: 1577.261952161789 second


In [26]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_pure_tfidf_160_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_pure_tfidf_160_bi.txt"
get_completion(prompt_list, save_path)

0  Non-functional
50  Non-functional
100  Functional
150  Functional
200  Non-functional
250  Functional
300  Non-functional
350  Functional
400  Non-functional
450  Functional
500  Non-functional
550  Functional
600  Functional
execution time: 1630.1769001483917 second


## multi-class classification

#### promise multi-class random

In [71]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_random_0.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_random_0.txt"
get_completion(prompt_list, save_path)

execution time: 103.16644406318665 second


In [72]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_random_5.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_random_5.txt"
get_completion(prompt_list, save_path)

execution time: 123.59215784072876 second


In [73]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_random_10.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_random_10.txt"
get_completion(prompt_list, save_path)

execution time: 151.8356590270996 second


In [74]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_random_20.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_random_20.txt"
get_completion(prompt_list, save_path)

execution time: 124.51837396621704 second


In [75]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_random_40.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_random_40.txt"
get_completion(prompt_list, save_path)

execution time: 122.07912182807922 second


In [76]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_random_80.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_random_80.txt"
get_completion(prompt_list, save_path)

execution time: 120.85491275787354 second


In [23]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_random_120.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_random_120.txt"
get_completion(prompt_list, save_path)

0  Functional
50  Look and Feel
100  Maintainability
execution time: 330.31263494491577 second


In [111]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_random_160.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_random_160.txt"
get_completion(prompt_list, save_path)

execution time: 123.72332906723022 second


#### promise multiclass embedding

In [77]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_embedding_0.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_embedding_0.txt"
get_completion(prompt_list, save_path)

execution time: 121.12382888793945 second


In [78]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_embedding_5.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_embedding_5.txt"
get_completion(prompt_list, save_path)

execution time: 121.81056785583496 second


In [79]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_embedding_10.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_embedding_10.txt"
get_completion(prompt_list, save_path)

execution time: 117.41837573051453 second


In [80]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_embedding_20.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_embedding_20.txt"
get_completion(prompt_list, save_path)

execution time: 125.62616109848022 second


In [81]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_embedding_40.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_embedding_40.txt"
get_completion(prompt_list, save_path)

execution time: 123.28952097892761 second


In [82]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_embedding_80.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_embedding_80.txt"
get_completion(prompt_list, save_path)

execution time: 125.4775390625 second


In [13]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_embedding_120.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_embedding_120.txt"
get_completion(prompt_list, save_path)

0  Functional
50  Look and Feel
96th prompt, Connection error: try again automatically 0 times
100  The given requirement can be classified as Maintainability. This is because the requirement ensures that the system maintains the data even after the customer cancels their account, which implies that the system is designed to handle data changes and updates over time.
execution time: 370.46543407440186 second


In [112]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_embedding_160.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_embedding_160.txt"
get_completion(prompt_list, save_path)

execution time: 134.59514784812927 second


#### promise multiclass tf-idf

In [83]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_tfidf_0.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_tfidf_0.txt"
get_completion(prompt_list, save_path)

execution time: 112.45754504203796 second


In [84]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_tfidf_5.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_tfidf_5.txt"
get_completion(prompt_list, save_path)

execution time: 120.38309717178345 second


In [85]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_tfidf_10.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_tfidf_10.txt"
get_completion(prompt_list, save_path)

execution time: 125.04896903038025 second


In [86]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_tfidf_20.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_tfidf_20.txt"
get_completion(prompt_list, save_path)

execution time: 115.85242700576782 second


In [87]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_tfidf_40.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_tfidf_40.txt"
get_completion(prompt_list, save_path)

execution time: 126.12441611289978 second


In [22]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_tfidf_80.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_tfidf_80.txt"
get_completion(prompt_list, save_path)

0  Functional
50  Look and Feel
100  Maintainability
execution time: 315.2939648628235 second


In [14]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_tfidf_120.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_tfidf_120.txt"
get_completion(prompt_list, save_path)

0  Functional
50  Look and Feel
100  Maintainability
execution time: 358.8730478286743 second


In [113]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_tfidf_160.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_tfidf_160.txt"
get_completion(prompt_list, save_path)

execution time: 127.57248735427856 second


## new promise multi-class evaluation|

In [9]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_new_random_0_mul.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_new_random_0.txt"
get_completion(prompt_list, save_path)

path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_new_random_5_mul.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_new_random_5.txt"
get_completion(prompt_list, save_path)

path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_new_random_10_mul.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_new_random_10.txt"
get_completion(prompt_list, save_path)

path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_new_random_20_mul.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_new_random_20.txt"
get_completion(prompt_list, save_path)

path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_new_random_40_mul.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_new_random_40.txt"
get_completion(prompt_list, save_path)

path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_new_random_80_mul.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_new_random_80.txt"
get_completion(prompt_list, save_path)

path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_new_random_120_mul.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_new_random_120.txt"
get_completion(prompt_list, save_path)

path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_new_random_160_mul.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_new_random_160.txt"
get_completion(prompt_list, save_path)

0  Functional Suitability
50  Functional Suitability
100  ["Functional Suitability"]
execution time: 328.46792697906494 second
0  Functional Suitability
50  Usability
100  Performance Efficiency
execution time: 302.65255665779114 second
0  Functional Suitability
50  Usability
100  Performance Efficiency
execution time: 310.7597596645355 second
0  Functional Suitability
50  Usability
100  Performance Efficiency
execution time: 339.11315298080444 second
0  Functional Suitability
50  Usability
100  Functional Suitability
execution time: 316.6370151042938 second
0  Functional Suitability
50  Functional Suitability
100  Functional Suitability
execution time: 403.83241605758667 second
0  Functional Suitability
50  Functional Suitability
100  Performance Efficiency
execution time: 351.9027750492096 second
0  Functional Suitability
50  Usability
100  Functional Suitability
execution time: 360.593621969223 second


In [10]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_new_embedding_0_mul.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_new_embedding_0.txt"
get_completion(prompt_list, save_path)

path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_new_embedding_5_mul.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_new_embedding_5.txt"
get_completion(prompt_list, save_path)

path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_new_embedding_10_mul.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_new_embedding_10.txt"
get_completion(prompt_list, save_path)

path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_new_embedding_20_mul.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_new_embedding_20.txt"
get_completion(prompt_list, save_path)

path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_new_embedding_40_mul.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_new_embedding_40.txt"
get_completion(prompt_list, save_path)

path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_new_embedding_80_mul.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_new_embedding_80.txt"
get_completion(prompt_list, save_path)

path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_new_embedding_120_mul.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_new_embedding_120.txt"
get_completion(prompt_list, save_path)

path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_new_embedding_160_mul.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_new_embedding_160.txt"
get_completion(prompt_list, save_path)

0  Functional Suitability
50  Functional Suitability
100  ["Functional Suitability"]
execution time: 300.6656630039215 second
0  Usability
50  Usability
100  Performance Efficiency
execution time: 302.2897379398346 second
0  Functional Suitability
50  Usability
100  Performance Efficiency
execution time: 304.63058710098267 second
0  Functional Suitability
50  Usability
100  Performance Efficiency
execution time: 311.7528429031372 second
0  Functional Suitability
50  Functional Suitability
100  Functional Suitability
execution time: 319.9521450996399 second
0  Functional Suitability
50  Functional Suitability
100  Functional Suitability
execution time: 337.20666790008545 second
0  Functional Suitability
50  Functional Suitability
100  Functional Suitability
execution time: 362.54729771614075 second
0  Functional Suitability
50  Functional Suitability
100  Functional Suitability
execution time: 378.770201921463 second


In [9]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_new_tfidf_0_mul.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_new_tfidf_0.txt"
get_completion(prompt_list, save_path)

path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_new_tfidf_5_mul.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_new_tfidf_5.txt"
get_completion(prompt_list, save_path)

path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_new_tfidf_10_mul.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_new_tfidf_10.txt"
get_completion(prompt_list, save_path)

path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_new_tfidf_20_mul.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_new_tfidf_20.txt"
get_completion(prompt_list, save_path)

path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_new_tfidf_40_mul.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_new_tfidf_40.txt"
get_completion(prompt_list, save_path)

path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_new_tfidf_80_mul.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_new_tfidf_80.txt"
get_completion(prompt_list, save_path)

path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_new_tfidf_120_mul.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_new_tfidf_120.txt"
get_completion(prompt_list, save_path)

path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_new_tfidf_160_mul.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_new_tfidf_160.txt"
get_completion(prompt_list, save_path)

0  Functional Suitability
50  Functional Suitability
100  ["Functional Suitability"]
execution time: 304.08283710479736 second
0  Functional Suitability
50  Usability
100  Functional Suitability
execution time: 305.22074794769287 second
0  Functional Suitability
50  Functional Suitability
100  Functional Suitability
execution time: 310.2280750274658 second
0  Functional Suitability
50  Usability
100  Functional Suitability
execution time: 305.2864329814911 second
0  Functional Suitability
50  Functional Suitability
100  Functional Suitability
execution time: 315.27965092658997 second
0  Functional Suitability
50  Functional Suitability
100  Functional Suitability
execution time: 335.75928592681885 second
0  Functional Suitability
50  Functional Suitability
100  Functional Suitability
execution time: 352.3476040363312 second
0  Functional Suitability
50  Functional Suitability
100  Functional Suitability
execution time: 396.0614290237427 second


# get completion on nfr dataset

## multi-class classification

#### nfr multi_class random

In [89]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_nfr_random_0.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_nfr_random_0.txt"
get_completion(prompt_list, save_path)

execution time: 3460.546215057373 second


In [97]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_nfr_random_5.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_nfr_random_5.txt"
get_completion(prompt_list, save_path)

execution time: 3478.666871070862 second


In [114]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_nfr_random_10.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_nfr_random_10.txt"
get_completion(prompt_list, save_path)

execution time: 3468.873160123825 second


In [115]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_nfr_random_20.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_nfr_random_20.txt"
get_completion(prompt_list, save_path)

execution time: 3486.1135108470917 second


In [116]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_nfr_random_40.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_nfr_random_40.txt"
get_completion(prompt_list, save_path)

execution time: 3484.3390600681305 second


In [117]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_nfr_random_80.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_nfr_random_80.txt"
get_completion(prompt_list, save_path)

execution time: 3487.0319340229034 second


In [ ]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_nfr_random_120.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_nfr_random_120.txt"
get_completion(prompt_list, save_path)

In [118]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_nfr_random_160.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_nfr_random_160.txt"
get_completion(prompt_list, save_path)

execution time: 27610.913210868835 second


#### nfr multi_class embedding

In [119]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_nfr_embedding_0.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_nfr_embedding_0.txt"
get_completion(prompt_list, save_path)

execution time: 9217.068017959595 second


In [120]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_nfr_embedding_5.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_nfr_embedding_5.txt"
get_completion(prompt_list, save_path)

execution time: 3708.531285047531 second


In [121]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_nfr_embedding_10.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_nfr_embedding_10.txt"
get_completion(prompt_list, save_path)

execution time: 3597.099881887436 second


In [122]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_nfr_embedding_20.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_nfr_embedding_20.txt"
get_completion(prompt_list, save_path)

execution time: 3647.655138015747 second


In [123]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_nfr_embedding_40.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_nfr_embedding_40.txt"
get_completion(prompt_list, save_path)

execution time: 3484.010708808899 second


In [8]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_nfr_embedding_80.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_nfr_embedding_80.txt"
get_completion(prompt_list, save_path)

execution time: 3463.9130859375 second


In [ ]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_nfr_embedding_120.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_nfr_embedding_120.txt"
get_completion(prompt_list, save_path)

In [9]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_nfr_embedding_160.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_nfr_embedding_160.txt"
get_completion(prompt_list, save_path)

execution time: 3492.20218706131 second


#### nfr multi_class tfidf

In [10]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_nfr_tfidf_0.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_nfr_tfidf_0.txt"
get_completion(prompt_list, save_path)

execution time: 3478.533807039261 second


In [11]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_nfr_tfidf_5.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_nfr_tfidf_5.txt"
get_completion(prompt_list, save_path)

execution time: 3480.6994071006775 second


In [12]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_nfr_tfidf_10.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_nfr_tfidf_10.txt"
get_completion(prompt_list, save_path)

execution time: 35353.59642314911 second


In [13]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_nfr_tfidf_20.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_nfr_tfidf_20.txt"
get_completion(prompt_list, save_path)

execution time: 3509.2056109905243 second


In [14]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_nfr_tfidf_40.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_nfr_tfidf_40.txt"
get_completion(prompt_list, save_path)

execution time: 3484.5739438533783 second


In [15]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_nfr_tfidf_80.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_nfr_tfidf_80.txt"
get_completion(prompt_list, save_path)

50th prompt. At least one prompt did not get the result.
execution time: 2838.492636203766 second


In [ ]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_nfr_tfidf_120.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_nfr_tfidf_120.txt"
get_completion(prompt_list, save_path)

In [16]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_nfr_tfidf_160.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_nfr_tfidf_160.txt"
get_completion(prompt_list, save_path)

50th prompt. At least one prompt did not get the result.
execution time: 251.89404487609863 second
